In [2]:
import re
import os
import json
import pprint
import pandas as pd

from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo import MongoClient, DESCENDING , UpdateOne
from pymongo.server_api import ServerApi
import hashlib

load_dotenv()

True

In [4]:
folder_path = '../dataRAW'
cars = []

for filename in os.listdir(folder_path):
    #print("Procesando archivo: ", filename)
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                #print(f'Hay {len(data)} coches')
                cars.extend(data)
            except json.JSONDecodeError as e:
                print(f"Error al procesar {filename}: {e}")

print(f'ARCHIVOS PROCESADOS: {len(os.listdir(folder_path))} TOTAL: {len(cars)}')

ARCHIVOS PROCESADOS: 1000 TOTAL: 196917


In [5]:
print(cars[:5])

[{'id': '58841374', 'creationDate': '2024-10-01T00:00:40Z', 'title': 'DACIA Dokker Van Essential dCi 55kW 75CV 4p.', 'url': '/dacia-dokker-van-essential-dci-55kw-75cv-diesel-2019-en-las_palmas-58841374-fuvivo.aspx', 'price': {'amount': 10990, 'taxTypeId': 2, 'hasTaxes': True}, 'seller': {'name': 'Shoping Car ', 'isProfessional': True, 'contractId': '79909', 'pack': {'legacyId': 21, 'type': 'advance'}}, 'km': 73500, 'year': 2019, 'cubicCapacity': 1461, 'provinceIds': [35], 'mainProvince': 'Las Palmas', 'location': {'provinceIds': [35], 'mainProvince': 'Las Palmas', 'mainProvinceId': 35, 'cityId': 35027, 'cityLiteral': 'Teror'}, 'resources': [{'type': 'IMAGE', 'url': 'https://a.ccdn.es/cnet/2024/10/01/58841374/1959911610_g.jpg'}, {'type': 'IMAGE', 'url': 'https://a.ccdn.es/cnet/2024/10/01/58841374/1959911638_g.jpg'}, {'type': 'IMAGE', 'url': 'https://a.ccdn.es/cnet/2024/10/01/58841374/1959911646_g.jpg'}, {'type': 'IMAGE', 'url': 'https://a.ccdn.es/cnet/2024/10/01/58841374/1959911652_g.jp

In [61]:
#import index.csv
df = pd.read_csv('../data/Index.csv', dtype={'car_id': 'int64'})
pd.options.display.float_format = '{:.0f}'.format
df_unique = df.drop_duplicates(subset='id', keep='first')

id_to_car_id = pd.Series(df_unique['idx'].values, index=df_unique['id']).to_dict()

# Comprobar que se está convirtiendo bien (imprimir las primeras 10 entradas)
for i, (id_value, idx) in enumerate(id_to_car_id.items()):
    if i < 10:  # Mostrar solo las primeras 10 entradas
        print(f"id: {id_value} -> idx: {idx}")
        
print(id_to_car_id[58841374])

id: 58594130 -> idx: 0
id: 58594037 -> idx: 1
id: 58707917 -> idx: 2
id: 56836987 -> idx: 3
id: 58629180 -> idx: 4
id: 57747053 -> idx: 5
id: 58043946 -> idx: 6
id: 58251203 -> idx: 7
id: 55368579 -> idx: 8
id: 58764390 -> idx: 9
0


In [64]:
car_image_dict = {}

for car in cars:
    resourcesClean = []
    for img in car['resources']:
        resourcesClean.append(img['url'])

    if resourcesClean != []:
        if int(car['id']) in id_to_car_id:
            idx = id_to_car_id[int(car['id'])]
            car_image_dict[idx] = resourcesClean

#preview dict
for i, (idx, images) in enumerate(car_image_dict.items()):
    if i < 10:  # Mostrar solo las primeras 10 entradas
        print(f"idx: {idx} -> images: {images}")
        
print(len(car_image_dict))

idx: 15671 -> images: ['https://a.ccdn.es/cnet/2024/09/20/58765577/1955277103_g.jpg', 'https://a.ccdn.es/cnet/2024/09/20/58765577/1955277105_g.jpg', 'https://a.ccdn.es/cnet/2024/09/20/58765577/1955277107_g.jpg', 'https://a.ccdn.es/cnet/2024/09/20/58765577/1955277109_g.jpg', 'https://a.ccdn.es/cnet/2024/09/20/58765577/1955277110_g.jpg', 'https://a.ccdn.es/cnet/2024/09/20/58765577/1955277127_g.jpg', 'https://a.ccdn.es/cnet/2024/09/20/58765577/1955277132_g.jpg', 'https://a.ccdn.es/cnet/2024/09/20/58765577/1955277137_g.jpg', 'https://a.ccdn.es/cnet/2024/09/20/58765577/1955277145_g.jpg', 'https://a.ccdn.es/cnet/2024/09/20/58765577/1955277150_g.jpg']
idx: 12137 -> images: ['https://a.ccdn.es/cnet/vehicles/16051462/79fd3409-1cb7-4b80-9c3d-3001e660d084.jpg', 'https://a.ccdn.es/cnet/vehicles/16051462/6d77c067-ceaa-4160-82ac-be272d5314ed.jpg', 'https://a.ccdn.es/cnet/vehicles/16051462/e8278926-d4ef-45af-8938-2786e8565fa8.jpg', 'https://a.ccdn.es/cnet/vehicles/16051462/a2bb735b-498a-4189-ae83-bbd

In [70]:
db_uri = os.getenv('MONGO_DB_URI')

# Create a new client and connect to the server
client = MongoClient(db_uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)
    
db = client.db
collection = db.cars
    
    

idx_with_resources = list(car_image_dict.keys())

# Consultar todos los coches que ya existen en la base de datos de una sola vez
existing_cars = [doc['_id'] for doc in collection.find({'_id': {'$in': idx_with_resources}})]



Pinged your deployment. You successfully connected to MongoDB!
39399


In [72]:
#Borrar todos los documentos que no esten en existing_cars
result = collection.delete_many({'_id': {'$nin': existing_cars}})
print(f"Documentos eliminados: {result.deleted_count}")

Documentos eliminados: 54883


In [73]:
# Crear una lista de objetos UpdateOne para actualizar los coches que ya existen
requests = []

for idx, images in car_image_dict.items():
    if idx in existing_cars:
        requests.append(
            UpdateOne(
                {'_id': idx},
                {'$set': {'images': images}}
            )
        )
    
# Enviar las peticiones de actualización a la base de datos
result = collection.bulk_write(requests)